In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-09-21 08:17:13.940275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 08:17:14.447807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [4]:
model = SAT1Topological(8, 5, len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

2023-09-21 08:17:17.714500: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-21 08:17:17.739626: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-21 08:17:17.739709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-21 08:17:17.741383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-21 08:17:17.741446: I tensorflow/compile

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 5, 154, 1)]    0         
                                                                 
 masking (Masking)           (None, 8, 5, 154, 1)      0         
                                                                 
 conv3d (Conv3D)             (None, 4, 3, 150, 64)     4864      
                                                                 
 max_pooling3d (MaxPooling3  (None, 4, 3, 75, 64)      0         
 D)                                                              
                                                                 


 conv3d_1 (Conv3D)           (None, 2, 1, 73, 128)     221312    
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 2, 1, 36, 128)     0         
 g3D)                                                            
                                                                 
 conv3d_2 (Conv3D)           (None, 2, 1, 34, 256)     98560     
                                                                 
 max_pooling3d_2 (MaxPoolin  (None, 2, 1, 17, 256)     0         
 g3D)                                                            
                                                                 
 flatten (Flatten)           (None, 8704)              0         
                                                                 
 dense (Dense)               (None, 128)               1114240   
                                                                 
 dropout (Dropout)           (None, 128)               0         
          

In [5]:
# model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#     metrics=["accuracy"],
# )

In [5]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    gen_kwargs={"shape_topological": True},
)

Epoch 1/20


2023-09-21 08:17:25.414648: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-21 08:17:26.324087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-21 08:17:26.326538: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5620c93575d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-21 08:17:26.326573: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-09-21 08:17:26.330269: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-21 08:17:26.426720: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

765/765 [==============================] - 14s 14ms/step - loss: 0.9947 - accuracy: 0.6018 - val_loss: 0.8098 - val_accuracy: 0.6982
Epoch 2/20
765/765 [==============================] - 11s 14ms/step - loss: 0.7357 - accuracy: 0.7191 - val_loss: 0.7066 - val_accuracy: 0.7450
Epoch 3/20
765/765 [==============================] - 10s 13ms/step - loss: 0.6505 - accuracy: 0.7545 - val_loss: 0.6497 - val_accuracy: 0.7634
Epoch 4/20
765/765 [==============================] - 11s 14ms/step - loss: 0.5829 - accuracy: 0.7794 - val_loss: 0.6072 - val_accuracy: 0.7819
Epoch 5/20
765/765 [==============================] - 10s 13ms/step - loss: 0.5430 - accuracy: 0.7984 - val_loss: 0.5668 - val_accuracy: 0.7953
Epoch 6/20
765/765 [==============================] - 11s 13ms/step - loss: 0.5098 - accuracy: 0.8109 - val_loss: 0.5606 - val_accuracy: 0.7871
Epoch 7/20
765/765 [==============================] - 11s 13ms/step - loss: 0.4821 - accuracy: 0.8226 - val_loss: 0.5322 - val_accuracy: 0.8043
Epo

(<keras.src.callbacks.History at 0x7fa4c16cad50>, None)

In [24]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 147, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 30, 143, 64)       384       
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 71, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 69, 128)       24704     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 34, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 32, 256)       98560 